In [ ]:
from openai import OpenAI

In [ ]:
#first we need to learn about selenium to understrand to scrape shopping sites 
!pip install selenium

In [ ]:
# Selenium renders the JS and the fetches the content from it 
# It uses pip install webdriver-manager to make he python code to UI management possible

!pip install webdriver-manager

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
service = Service(ChromeDriverManager().install())



In [ ]:
def scrapper_amazon(product_name):
  driver = webdriver.Chrome(service=service)
  driver.get("https://www.amazon.in/")
  try:
    search_bar_locator = (By.ID,"twotabsearchtextbox")
    search_button_locator = (By.ID, "nav-search-submit-button")

    search_bar = WebDriverWait(driver,10).until(EC.presence_of_element_located(search_bar_locator))

    search_bar.send_keys(product_name)
    driver.find_element(*search_button_locator).click()
    print(f"{product_name}")
    # Get the first product card element
    first_product = driver.find_elements(By.XPATH, "//div[@class='puisg-row']")
    first_product = [e for e in first_product if e]
    top5 = first_product[:5]
    top_5_text = [e.text for e in top5]
    driver.quit()
    return top_5_text

  except Exception as e:
    print(e)
    driver.quit()




In [ ]:


def scrapper_flipkart(product_name):
    driver = webdriver.Chrome(service=service)
    driver.get("https://www.flipkart.com/")

    try:
        try:
            close_btn = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, "//span[text()='✕']"))
            )
            close_btn.click()
        except:
            pass  # popup didn't appear

        # --- 2. Find search box ---
        search_bar_locator = (By.NAME, "q")
        search_bar = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(search_bar_locator)
        )
        search_bar.send_keys(product_name)
        search_bar.submit()

        print(f"Searching Flipkart for: {product_name}")
        time.sleep(3)

       
        product_cards = driver.find_elements(By.XPATH, "//div[contains(@class, 'yKfJKb')]")

        top5 = product_cards[:5]
        
        top5_text = [card.text for card in top5]
        driver.quit()
        return top5_text

    except Exception as e:
        print("Error:", e)
        driver.quit()


In [ ]:
def scrape_sites(search_product):
  amaL = scrapper_amazon(search_product)
  FlipL = scrapper_flipkart(search_product)
  
  return {"amazonR":amaL,"flipkartR":FlipL}


In [ ]:
def content_call(product_name):
  results = scrape_sites(product_name)
  amazonR = results["amazonR"]
  flipkartR = results["flipkartR"]
  print(amazonR)
  amazon_newline = "\n".join(amazonR)
  print(amazon_newline)
  flipkart_newline = "\n".join(flipkartR)
  userContent = f"""
  Here are the options of amazon for the {product_name} search : 
  {amazon_newline}

  Flipkart:
  {flipkart_newline}

  Compare them both and let me know who offer better offer 
  
  make sure to take the discounted price while comparing leave out the MRP
  """
  return userContent


In [ ]:
def messages_sysNuser(product_name):
  system_prompt ="""
  You are a product comparison AI.

Your job:
- Compare product prices from Amazon and Flipkart.
- Use ONLY the discounted price (ignore MRP).
- Extract a clean comparison table.
- Output MUST follow this structure:

### 🛒 Product Comparison Summary
### 📦 Amazon Listings
### 📦 Flipkart Listings
### 📊 Extracted Comparison Table
### 🏆 Final Recommendation

Format EVERYTHING in clean Markdown.
No disclaimers. No extra text.

  """
  return [
    {"role":"system","content":system_prompt},
    {"role":"user","content":content_call(product_name)}
  ]

In [ ]:
from openai import api_key


def search_product(product_name):
  openai = OpenAI(api_key=api_key)
  response = openai.chat.completions.create(
    model = "gpt-4.1-nano",
    messages = messages_sysNuser(product_name)
  )
  return response.choices[0].message.content

In [ ]:
from IPython.display import Markdown, display
product_name ="Boat Rockerz 650"
display(Markdown(search_product(product_name)))
